In [1]:
import pandas as pd
import pymongo

In [2]:
df = pd.read_csv("../Ryan_Cleaned_Data/CombinedTotal.csv")

c:\users\mnyon\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df = df.drop_duplicates(subset = ["Description", "Country", "Province"], keep = "first").drop_duplicates(subset = ["Description", "Country"], keep = "first")

In [5]:
len(df)

169435

In [6]:
(df[["Winery", "Country"]].groupby("Winery")["Country"].nunique() > 1).value_counts()

False    18975
True       205
Name: Country, dtype: int64

In [7]:
(df[["Winery", "Province"]].groupby("Winery")["Province"].nunique() > 1).value_counts()

False    17358
True      1822
Name: Province, dtype: int64

In [8]:
# Combine region_1 and region_2 into a single column using lists
region_list = []

for thing in df.iterrows():
    region_1 = str(list(thing)[1].loc["Region_1"])
    region_2 = str(list(thing)[1].loc["Region_2"])

# list containing the number of region entries for each row
    if region_2 == "nan":
        if region_1 == "nan":
            region = []
        else:
            region = [region_1]
    else:
        region = [region_1, region_2]

    region_list.append(region)

df["Region"] = region_list

In [9]:
wcp = ["Winery", "Country", "Province"]

In [10]:
# build dataframe of info for each winery
winery_df = pd.DataFrame()

In [11]:
# mean points and prices
winery_df[["Points_Mean", "Price_Mean"]] = df[["Winery", "Country", "Province", "Points", "Price"]].groupby(wcp).mean().round(1)

In [12]:
# median points and prices
winery_df[["Points_Median", "Price_Median"]] = df[["Winery", "Country", "Province", "Points", "Price"]].groupby(wcp).median()

In [13]:
# total number of reviews
winery_df["Total_Reviews"] = df[["Winery", "Country", "Province", "Description"]].groupby(wcp).count()

In [14]:
# count the number of different wine varieties
winery_df["N_Varieties"] = df[["Winery", "Country", "Province", "Variety"]].groupby(wcp).nunique()

In [15]:
# list of each type of wine variety
wine_types = df[["Winery", "Country", "Province", "Variety"]].groupby(wcp)["Variety"].unique()
wine_list = []

for item in wine_types:
    wines = item.tolist()
    wine_list.append(wines)
    
winery_df["Varieties"] = wine_list

In [16]:
# region data
winery_df["Region"] = df[["Winery", "Country", "Province", "Region"]].groupby(wcp).first()

In [17]:
winery_df.head()

,,,Points_Mean,Price_Mean,Points_Median,Price_Median,Total_Reviews,N_Varieties,Varieties,Region
Winery,Country,Province,,,,,,,,
'37 Cellars,US,Washington,88.0,NaN,88.0,NaN,1,1,[Cabernet Sauvignon],"[Columbia Valley (WA), Columbia Valley]"
1+1=3,Spain,Catalonia,85.5,16.3,86.0,16.0,10,3,"[Rosé, Cabernet Sauvignon, Sparkling Blend]",[Penedès]
10 Knots,US,California,84.4,23.3,84.0,22.0,11,7,"[Grenache, Rhône-style White Blend, Chardonnay...","[Paso Robles, Central Coast]"
100 Percent Wine,US,California,86.3,18.0,86.0,18.0,3,3,"[Moscato, Sauvignon Blanc, Red Blend]","[California, California Other]"
1000 Stories,US,California,90.5,19.0,90.5,19.0,2,1,[Zinfandel],"[California, California Other]"


In [18]:
len(winery_df)

22123

In [19]:
# add winery name, country, and province as columns instead of indices
winery_df = winery_df.reset_index()

In [21]:
# column rearrangement
cols = ['Winery', 'Points_Mean', 'Price_Mean', 'Points_Median', 'Price_Median', 'Total_Reviews', 'N_Varieties', 'Varieties', 'Country', 'Province', 'Region']
winery_df = winery_df[cols]

In [22]:
winery_df.head()

,Winery,Points_Mean,Price_Mean,Points_Median,Price_Median,Total_Reviews,N_Varieties,Varieties,Country,Province,Region
0,'37 Cellars,88.0,NaN,88.0,NaN,1,1,[Cabernet Sauvignon],US,Washington,"[Columbia Valley (WA), Columbia Valley]"
1,1+1=3,85.5,16.3,86.0,16.0,10,3,"[Rosé, Cabernet Sauvignon, Sparkling Blend]",Spain,Catalonia,[Penedès]
2,10 Knots,84.4,23.3,84.0,22.0,11,7,"[Grenache, Rhône-style White Blend, Chardonnay...",US,California,"[Paso Robles, Central Coast]"
3,100 Percent Wine,86.3,18.0,86.0,18.0,3,3,"[Moscato, Sauvignon Blanc, Red Blend]",US,California,"[California, California Other]"
4,1000 Stories,90.5,19.0,90.5,19.0,2,1,[Zinfandel],US,California,"[California, California Other]"


In [23]:
# create mongo database
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.winery_db

In [24]:
# convert winery dataframe to dictionary
winery_dict = winery_df.fillna("").to_dict(orient = "index")

In [25]:
# insert dictionary into winery_db.data
db.data.insert_many(winery_dict.values())

In [26]:
winery_df.to_csv("wineries.csv")

In [27]:
winery_df[["Winery", "Province"]].value_counts() > 1

Winery              Province            
'37 Cellars         Washington              False
Mount Michael       Central Otago           False
Mount St. Helena    California              False
Mount Si            Washington              False
Mount Riley         South Island            False
                                            ...  
Domaine Belland     Burgundy                False
Domaine Begude      Languedoc-Roussillon    False
Domaine Becquet     California              False
Domaine Beauséjour  Loire Valley            False
Štoka               Kras                    False
Length: 22123, dtype: bool

In [28]:
winery_df[["Winery", "Country"]].value_counts() > 1

Winery                          Country 
Undurraga                       Chile        True
Concha y Toro                   Chile        True
Santa Carolina                  Chile        True
Casca Wines                     Portugal     True
Valdivieso                      Chile        True
                                            ...  
Domaine Coquard Loison-Fleurot  France      False
Domaine Combe Blanche           France      False
Domaine Collotte                France      False
Domaine Clusel-Roch             France      False
Štoka                           Slovenia    False
Length: 19442, dtype: bool